### Comprehension Burden for Sequencing Documents

In [1]:
import requests
from collections import Counter
import random as randomlib

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import networkx as nx

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
'''
Knowledge Graph
'''
kg_path = "../graph_query/graphs/knowledge_graph.gpickle"
kg = nx.read_gpickle(kg_path)
kg_labels = [str(x) for x in list(kg.nodes())[1:]]
n_labels = len(kg_labels)

In [3]:
def get_queries_based_on_node(node_label):
    node = kg.node[node_label]

    if("NodeType" not in node): 
        kg.node[node_label]["NodeType"] = "ConceptNode"
        l =  list(kg.neighbors(node_label))
        return l
    node_type = node["NodeType"]
    if(node_type == "TopicNode" or node_type == "ConceptNode"):
        return list(kg.neighbors(node_label))
    elif(node_type == "SubConceptNode"):
        return [node_label]
    else:
        pass


    '''
    Returns a list of queries depending on the type of the
    node closest to the query.
    args - query(str)
    returns [] of str
    '''
def query_formulator(query, label):
    queries = []
    children_neighbours = get_queries_based_on_node(label)
    queries = [label]
    for child in children_neighbours:
        queries.append(child)
    return list(set(queries))


In [4]:
'''
Get content from a given set of URLs.
'''
es_order = []
def get_content():
    f = open('user_study_dynamic_programming_engage.txt', 'r')
    l = f.readlines()
    docs = {}
    index = {}
    
    counter = 0
    for url in l:
        try:
            docs[url] = requests.get(url).content
            index[url] = counter
            es_order.append(url)
            counter += 1
        except:
            continue
    return docs, index

In [5]:
'''
Term Frequency Array for a particular document.
'''
def get_tfd(content):
    word_count_dict = Counter(w for w in kg_labels 
                              if w.lower() in content.lower())
    common = word_count_dict.most_common()
    
    frequency_arr = [0]*len(kg_labels)
    
    for common_word in common:
        common_word_index = kg_labels.index(common_word[0])
        frequency_arr[common_word_index] = common_word[1]
    return frequency_arr

In [6]:
content, index = get_content()

In [7]:
'''
Building word_data a document (rows) by term frequency (columns) matrix.
'''
tfd_data = {}
for url, cont in content.items():
    tfd_data[url] = get_tfd(cont)

tfd_arr = []
for key in index.keys():
    tfd_arr.append(key.replace("\n", ""))

word_data = {'TFD':tfd_arr}

for label in kg_labels:
    word_data[label] = [None]*len(index)

for url, words_in_doc in tfd_data.items():
    url_index = index[url]
    for i in range(0, n_labels, 1):
        word = kg_labels[i]
        word_data[word][url_index] = words_in_doc[i]

In [8]:
'''
(DTF)^T(DTF) = Coocurence Matrix
'''
document_term_frequency = pd.DataFrame(word_data).set_index('TFD')
dtf_asint = document_term_frequency.astype(int)
coocc = dtf_asint.T.dot(dtf_asint)

### Calculating Relationship Score: S(i, j)

In [9]:
def get_relationship_between_concepts(concept_1, concept_2):
    concept_1_index= document_term_frequency.columns.get_loc(concept_1)
    concept_2_index= document_term_frequency.columns.get_loc(concept_2)
    
    return coocc.iloc[concept_1_index, concept_2_index]

### Significance of a concept in a document: \lambda(c, i)

In [10]:
def get_significance_score(concept, document):
    if(document == None): return 0
    concept_index = document_term_frequency.columns.get_loc(concept)
    freq = dtf_asint.iloc[index[document]][concept_index]
    coocc_row = coocc.iloc[concept_index,:] 
    r = np.array(coocc_row)
    if(sum(r) == 0): return 0
    return (freq)*np.count_nonzero(r)

In [11]:
process_labels = ['Processes', 'Process Creation', 'Implementation of Processes', 'Modeling Multiprogramming', 'Process States','Process Termination']
clustering_labels = ['Clustering', 'Spectral Clustering', 'K-Means', 'Unsupervised Learning']
sorting_labels = ['Sorting']
greedy_labels = ['Greedy Algorithms', 'Activity selection problem', 'Huffman codes']
divide_and_conquer_labels = ['Divide and Conquer', 'Substitution method solving recurrences', 'Master method solving recurrences', u'Dfs', 
     "Strassen's matrix multiplication",'Maximum subarray', 'Recursion-tree method solving recurrences', u'Pca']
graph_theory_labels = ["Graph Theory", "Topological Sort", "Strongly Connected Components",
                      "Depth-first search DFS", "Topological Sort"]
unsupervised_labels = ['Unsupervised Learning', u'Clustering', u'Spectral Clustering', 'K-means clustering', u'K-Means']
supervised_learning_labels = ['Supervised Learning', 'Linear Algebra', 'Vectorization', 'Naive Bayes', 'Gaussian Discriminant Analysis', 
     'Logistic Regression', 'Support Vector Machines', 'Perceptron', 'Genearting Learning Algorithms']
threads_labels = ['Threads', 'POSIX Threads', u'Bfs', 'Thread usage', 'Implementing Threads in User Space', 'Implementing Threads in the Kernel', 
                  'The Classical Thread Model', u'Kernel Methods', u'Memory Management']
dp_labels = ["Dynamic Programming", "Greedy Algorithms", "Dijkstra's algorithm"]

### Key Sections k_c

In [2]:
relevant_concepts = set()

key_doc = {}

doc_to_key = {}

labs = []
q = dp_labels
for each in q:
    labs.extend(query_formulator(each, each))

for each_document in content.keys():
    rt = []
    rc = []
    for each_concept in labs:
        s = get_significance_score(each_concept, each_document)
        if(s > 0):
            if("NodeType" not in kg.node[each_concept]):
                kg.node[each_concept]["NodeType"] = "ConceptNode"
                rc.append((each_concept, s))
            elif(kg.node[each_concept]["NodeType"] != "TopicNode" and 
               kg.node[each_concept]["NodeType"] != "SubjectNode" and
              kg.node[each_concept]["NodeType"] != "CourseNode"):
                rc.append((each_concept, s))
            else:
                if(kg.node[each_concept]["NodeType"] == "TopicNode"):
                    rt.append((each_concept, s))
    rt.sort(key=lambda x: x[1])
    rt = rt[::-1]
    
    rc.sort(key=lambda x: x[1])
    rc = rc[::-1]
    if(len(rc)):
        key_doc[each_document] = rc[0][0]
        relevant_concepts.add(rc[0][0])
        rc.pop(0)
        counter = 0
        while(counter < 10 and len(rc)):
            relevant_concepts.add(rc[0][0])
            rc.pop(0)
            counter += 1
    else:
        if(len(rt)):
            key_doc[each_document] = rt[0][0]
            relevant_concepts.add(rt[0][0])

NameError: name 'dp_labels' is not defined

### Comprehension Burden

In [16]:
def f_cb(sig_score, key_sig_score, relationship):
    return key_sig_score

def get_cb_document(document, document_key_concept, visited):
    key_sig_score = get_significance_score(document_key_concept, document)
    document_burden = 0.0
    num_of_docs = 0
    
    order = list(relevant_concepts)
    
    for other_concept in order:
        if(other_concept in visited or other_concept==document_key_concept): continue
        sig_score = get_significance_score(other_concept, document)
        relationship = get_relationship_between_concepts(document_key_concept, other_concept)
        if(sig_score > 0): 
            document_burden += f_cb(sig_score, key_sig_score, relationship)
            num_of_docs += 1
    return document_burden

In [17]:
print(relevant_concepts)

set(['Activity selection problem', 'Dynamic Programming', 'Rod cutting', 'Optimal binary search trees', "Dijkstra's algorithm", 'Huffman codes', 'Elements of dynamic programming', 'Longest common subsequence'])


### Sequence Generation

In [1]:
def get_linear(nodes):
    parents = []
    for each in nodes:
        if(kg.nodes[each]["NodeType"] == "TopicNode"):
            parents.append(each)
    
    linear = []
    for p in parents:
        linear.append(p)
        children = kg.neighbors(p)
        for c in children:
            if c in nodes and kg.nodes[c]["NodeType"] == "ConceptNode":
                linear.append(c)
    
    for each in nodes:
        if each not in linear:
            linear.append(each)
    
    return linear

def get_bfs(nodes):
    return []

def get_random(nodes):
    vals = list(nodes)
    r = []
    while(vals):
        s = randomlib.choice(vals)
        r.append(s)
        vals.remove(s)
    return r

def get_es(nodes, key_doc):
    order =[]
    for each in es_order:
        each = each.replace("\n", "")
        
        if(each in key_doc and key_doc[each] not in order):
            order.append(key_doc[each])
    for each in nodes:
        if (each not in order):
            order.append(each)
    return order
    
def get_sequences(nodes, key_doc):
        random = get_es(nodes, key_doc)
        linear = get_linear(nodes)
        top_down = linear[::-1]
        return random, linear, top_down

In [19]:
def get_cp_score(order, doc_to_key):
    visited = set()
    total = 0
    ordered = {}

    for each in order: ordered[each] = []
    for doc, kc in doc_to_key.items():
        ordered[kc].append(doc)

    for kc in order:
        visited.add(kc)
        docs = ordered[kc]
        for each in docs:
            total += get_cb_document(each, kc, visited)
    print(total)
    

In [20]:
es, linear, bottom_up = get_sequences(relevant_concepts, key_doc)

get_cp_score(es, key_doc)
get_cp_score(linear, key_doc)
get_cp_score(bottom_up, key_doc)

['Longest common subsequence']
969.0
399.0
830.0
